# Disaggregating Sigma

While keeping the CO2 emissions as is, we disagregate the sigma parameter by introducing Primary Energy. 

In [1]:
import pandas as pd

In [2]:
# Load BP Dataset
bp_dataset = pd.read_excel('bp-stats-review-2022-all-data.xlsx', sheet_name='Primary Energy Consumption', header=2, index_col=0)
bp_dataset.index.name = None

# Select years from base year until today
bp_years = [2005,2015] # bp_years = list(range(2005, 2022, 1))
bp_dataset = bp_dataset[bp_years]

# Drop rows with all NaN 
bp_dataset_clean = bp_dataset.dropna(axis = 0, how = 'all')

In [3]:
# Conver Exajoules (from BP Dataset) to kWh: 1 EJ = 2,778e+11 kWh
bp_dataset_clean = bp_dataset_clean * 2.778e+11

In [4]:
# Aggregating the Primary Energy Data per region

'''
Based on the EIA, a reference from the International Energy Outlook defines regions such as OECD-Europe, Other non-OECD Europe and Eurasia,
Other non-OECD Asia and the Middle East. We will be using these definitions moving forward. Also, Israel is included in OECD Europe for 
statistical reporting purposes (https://www.eia.gov/outlooks/ieo/pdf/IEO2021_ChartLibrary_References.pdf).

For OHI, High Income Countries were based on the European Commission data 
(https://ec.europa.eu/eurostat/web/international-statistical-cooperation/higher-income-countries). Questions about whether Signapore should 
be included in this regional category.
'''

# Aggregates from BP Dataset based on EIO
Latin_America_BP = ['Mexico','Total S. & Cent. America']
OHI_BP = ['Canada','Australia','New Zealand', 'South Korea'] 
OECD_Europe_BP = ['Austria', 'Belgium', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland',
'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom']
Middle_East_BP = ['Iran', 'Iraq', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 
'Other Middle East']
Non_OECD_Asia_BP = ['Bangladesh', 'China Hong Kong SAR', 'Indonesia',
'Malaysia', 'Pakistan', 'Philippines', 'Singapore', 'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam', 'Other Asia Pacific']
Non_Russia_Eurasia_BP = ['Azerbaijan', 'Belarus', 'Bulgaria', 'Croatia', 'Cyprus','Kazakhstan', 'North Macedonia',
'Romania','Turkmenistan', 'Ukraine','Uzbekistan', 'Other CIS']

# Single Country Regions
US_PE = bp_dataset_clean[bp_dataset_clean.index == 'US']
Russia_PE = bp_dataset_clean[bp_dataset_clean.index == 'Russian Federation'].rename(index={'Russian Federation':'Russia'}) 
China_PE = bp_dataset_clean[bp_dataset_clean.index == 'China']
Japan_PE = bp_dataset_clean[bp_dataset_clean.index == 'Japan']
India_PE = bp_dataset_clean[bp_dataset_clean.index == 'India']

# Simple Grouped Regions
Africa_PE = bp_dataset_clean[bp_dataset_clean.index == 'Total Africa'].rename(index = {'Total Africa':'Africa'})

Latin_America_PE = bp_dataset_clean.loc[Latin_America_BP]
Latin_America_PE.loc['Latin America'] = Latin_America_PE.sum()
Latin_America_PE = Latin_America_PE.drop(index = Latin_America_BP)

OHI_PE = bp_dataset_clean.loc[OHI_BP]
OHI_PE.loc['OHI'] = OHI_PE.sum()
OHI_PE = OHI_PE.drop(index = OHI_BP)

# Complex Grouped Regions

OECD_Europe_PE = bp_dataset_clean.loc[OECD_Europe_BP]
OECD_Europe_PE.loc['OECD-Europe'] = OECD_Europe_PE.sum()
OECD_Europe_PE = OECD_Europe_PE.drop(index = OECD_Europe_BP)

Middle_East_PE = bp_dataset_clean.loc[Middle_East_BP] 
Middle_East_PE.loc['Middle East'] = Middle_East_PE.sum()
Middle_East_PE = Middle_East_PE.drop(index = Middle_East_BP)

Non_OECD_Asia_PE = bp_dataset_clean.loc[Non_OECD_Asia_BP] 
Non_OECD_Asia_PE.loc['Other non-OECD Asia'] = Non_OECD_Asia_PE.sum()
Non_OECD_Asia_PE = Non_OECD_Asia_PE.drop(index = Non_OECD_Asia_BP)

Non_Russia_Eurasia_PE = bp_dataset_clean.loc[Non_Russia_Eurasia_BP]
Non_Russia_Eurasia_PE.loc['Non-Russia Eurasia'] = Non_Russia_Eurasia_PE.sum()
Non_Russia_Eurasia_PE = Non_Russia_Eurasia_PE.drop(index = Non_Russia_Eurasia_BP)

In [5]:
# Build one Primary Energy Dataframe
frames = [US_PE, OECD_Europe_PE, Japan_PE, Russia_PE, Non_Russia_Eurasia_PE, China_PE, India_PE, Middle_East_PE, Africa_PE, Latin_America_PE,
OHI_PE, Non_OECD_Asia_PE]

BP_Primary_Energy = pd.concat(frames)